In [1]:
using Rotations, SphericalHarmonics, WignerD, NFFT, LinearAlgebra, StaticArrays, StatsBase
using PyPlot, LaTeXStrings
include("../S2Point.jl")
include("../FindRotation.jl")
include("S2PlotSupport.jl")
include("figsize.jl")
plt.style.use("tex.mplstyle")
matplotlib.use("pgf");

In [2]:
n = 30
B = 15

create_grid(grids) = vec(collect.(Iterators.product(grids...)))
Rotationgrid = create_grid([range(0,2*pi, n), range(0,pi, n), range(0,2*pi, n)])

Wigner_Matrices = [[(wignerD(b, R...)) for b in 0:B] for R in Rotationgrid];

In [3]:
N = 3
U = [S2Point(.25*pi, 0.6*pi), S2Point(.7*pi, 1.35*pi), S2Point(.15*pi, .75*pi)]
R = RotZYZ(4,.7,1)
V = R * U;

In [4]:
wigner_values = real.(wigner_eval(V, U, B, Wigner_Matrices));

In [5]:
normed_wigner_values = (wigner_values .- minimum(wigner_values)) ./ (maximum(wigner_values) - minimum(wigner_values));

threshold = .33
index_of_wigner_above_threshold = findall(x -> x > threshold, normed_wigner_values)

wigner_above_threshold = wigner_values[index_of_wigner_above_threshold]
normed_wigner_values_above_threshold = normed_wigner_values[index_of_wigner_above_threshold]
Rotationgrid_above_threshold = Rotationgrid[index_of_wigner_above_threshold]

Rotationgrid_above_threshold_array = hcat(Rotationgrid_above_threshold...)
alpha, beta, gamma = (Rotationgrid_above_threshold_array[1,:], Rotationgrid_above_threshold_array[2,:], Rotationgrid_above_threshold_array[3,:]);

renormed_wigner_values_above_threshold = (normed_wigner_values_above_threshold .- threshold) ./ (1 - threshold)
sizing = renormed_wigner_values_above_threshold .* 8 .+ .5;

In [6]:
I = [[1,3], [2,3], [1,2]]

CrossingAngles = zeros(4,3)

for k in 1:3
    i = I[k]
    curve1 = find_support(U[i[1]],V[i[2]],500)
    curve2 = find_support(U[i[2]],V[i[1]],500)

    Approxmat = [norm(c1 - c2) <.01 for c1 in curve1, c2 in curve2]
    Index = findall(x -> x == 1, Approxmat)

    CrossingAngles[k,:] = transpose(curve1[Index[1][1]])
end

curve1 = find_support(U[1],V[1],500)
curve2 = find_support(U[2],V[2],500)

Approxmat = [norm(c1 - c2) <.01 for c1 in curve1, c2 in curve2]
Index = findall(x -> x == 1, Approxmat)

CrossingAngles[4,:] = transpose(curve1[Index[1][1]]);

In [7]:
matplotlib.pyplot.close()

fig = figure(figsize=(size_pt()));
gridsp = fig.add_gridspec(2, 2, height_ratios=[12, 1])
ax1 = fig.add_subplot(gridsp[1,1], projection="3d")
ax2 = fig.add_subplot(gridsp[1,2], projection="3d")
ax3 = fig.add_subplot(gridsp[2,1])
ax4 = fig.add_subplot(gridsp[2,2])

scatterplot = ax1.scatter(alpha, beta, gamma, c=wigner_above_threshold, s=2, cmap="inferno")

colorbar = fig.colorbar(scatterplot, ax=ax3, location="bottom", fraction=.53, aspect=16, orientation=:horizontal)
colorbar.set_label(latexstring("\$S_{$(B)}[\\nu(\\omega) \\ast \\mu(\\omega)]\$"))


draw_support(U, V, 500, ax2);
ax2.scatter(CrossingAngles[:,1], CrossingAngles[:,2], CrossingAngles[:,3], s=20, alpha=1, color=:black)

ax4.legend(ax2.get_legend_handles_labels()..., loc="upper center", ncol = 3, handlelength=1, borderaxespad=.85, columnspacing=0.5, handletextpad=.2)

for ax in [ax1, ax2]
    ax.set_xlabel(L"\alpha")
    ax.set_ylabel(L"\beta")
    ax.set_zlabel(L"\gamma")

    ax.set_xlim(0, 2*pi)
    ax.set_ylim(0, pi)
    ax.set_zlim(0, 2*pi)

    ax.xaxis.labelpad = -6
    ax.yaxis.labelpad = -4
    ax.zaxis.labelpad = -10

    ax.tick_params(axis="x", pad=-3)
    ax.tick_params(axis="y", pad=-3)
    ax.tick_params(axis="z", pad=-3)
end
ax3.axis("off")
ax4.axis("off")

fig.tight_layout()
fig.subplots_adjust(left=-.015, right=0.97, top=1.085, bottom=0.165, wspace=0.04)

fig.savefig("Output/Partialsum of Convolution and Support for 4 Points.pdf")